In [ ]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 788.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 22.0 MB/s eta 0:00

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.0+cu116 and Flower 1.1.0


## Data Preparation


In [ ]:
# parameters
time_step = 48
BATCH_SIZE = 32

In [ ]:
data = []
with open ('House_30.txt', 'r') as reader:
  for line in reader:
    stripped_line = line.strip().split()
    data.append(stripped_line)

tem = [x[0] for x in data]
houses = list(set(tem))

date = []
consumption = []
for i in houses:
  date.append([float(x[1]) for x in data if x[0]==i])
  consumption.append([float(x[2]) for x in data if x[0]==i])    

In [ ]:
def create_label(data, time_step):
  x_nest, y_nest = [], []
  for j in range(len(data)):
    x_data, y_data = [], []
    for i in range(len(data[j]) - time_step):
      x = data[j][i: (i + time_step)]
      x_data.append(x)
      y = [data[j][i + time_step]]
      y_data.append(y)

    #x_data = np.array(x_data)[:, :, np.newaxis]
    x_data = np.array(x_data)[:, :]
    #x_data = np.array(x_data)[:, np.newaxis, :]
    x_nest.append(x_data)
    y_nest.append(y_data)
  x_nest = np.array(x_nest)
  y_nest = np.array(y_nest)
  return x_nest, y_nest
# 可能要去掉x的最后一个维度 从（48，1）变为（48）

In [ ]:
input, labels = create_label(consumption, time_step)


In [ ]:
input = np.float32(input)
labels = np.float32(labels)


### Create Dataset

In [ ]:
# 定义GetLoader类，继承Dataset方法，并重写__getitem__()和__len__()方法
class GetLoader(torch.utils.data.Dataset):
	# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一脸懵逼
    def __len__(self):
        return len(self.data)


In [ ]:
length = len(input[0])
val = int(0.7*length)
test = int(0.9*length)
trainloaders = []
valloaders = []
testloaders = []

def load_datasets(input, labels):

  Xtrain_raw = [x[0: val] for x in input]
  Xval_raw = [x[val: test] for x in input]
  Xtest_raw = [x[test: ] for x in input]

  Ytrain_raw = [x[0: val] for x in labels]
  Yval_raw = [x[val: test] for x in labels]
  Ytest_raw = [x[test: ] for x in labels]

  for i in range(30):
    ds_train = GetLoader(Xtrain_raw[i], Ytrain_raw[i])
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    ds_val = GetLoader(Xval_raw[i], Yval_raw[i])
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    ds_test= GetLoader(Xtest_raw[i], Ytest_raw[i])
    testloaders.append(DataLoader(ds_test, batch_size=BATCH_SIZE))

  return trainloaders, valloaders, testloaders




In [ ]:
trainloaders, valloaders, testloaders = load_datasets(input, labels)

## Create Model

In [ ]:
import torch.nn as nn


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Linear(48, 48),
            act(),
            nn.Linear(48, 48),
            act(),
            nn.Linear(48, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = self.body(x)
        return out

net = Net().to(DEVICE)


criterion = nn.MSELoss() 

In [ ]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for x, y in trainloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(x)
            loss = criterion(net(x), y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
           
        epoch_loss /= len(trainloader.dataset)
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}") 


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = nn.MSELoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = net(x)
            loss += criterion(outputs, y).item()

    loss /= len(testloader.dataset)
   
    return loss 

### central test

In [ ]:
x, y = next(iter(trainloaders[0]))

In [ ]:
trainloader = trainloaders[0]
valloader = valloaders[0]
testloader = testloaders[0]
net = Net().to(DEVICE)


for epoch in range(5):
    train(net, trainloader, 1)
    loss = test(net, valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}")

loss = test(net, testloader)
print(f"Final test set performance:\n\tloss {loss}")



Epoch 1: validation loss 0.0005272865817465782
Epoch 2: validation loss 0.0005215112082921317
Epoch 3: validation loss 0.0004672112590357708
Epoch 4: validation loss 0.000467944144985038
Epoch 5: validation loss 0.00046927032099544966
Final test set performance:
	loss 0.00042405893019403437


## FL

In [ ]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)



In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(0)}



In [ ]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)



    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [ ]:
NUM_CLIENTS = 30

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.3,  # Sample 50% of available clients for evaluation
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
        min_available_clients=10,  # Wait until all 10 clients are available
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
  client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5), #10
    strategy=strategy,
    client_resources=client_resources,
)

INFO flower 2023-01-13 15:49:57,209 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO:flower:Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
2023-01-13 15:49:59,925	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2023-01-13 15:50:01,494 | app.py:174 | Flower VCE: Ray initialized with resources: {'node:172.28.0.2': 1.0, 'memory': 7535598798.0, 'object_store_memory': 3767799398.0, 'CPU': 2.0}
INFO:flower:Flower VCE: Ray initialized with resources: {'node:172.28.0.2': 1.0, 'memory': 7535598798.0, 'object_store_memory': 3767799398.0, 'CPU': 2.0}
INFO flower 2023-01-13 15:50:01,506 | server.py:86 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2023-01-13 15:50:01,514 | server.py:270 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
INFO flower 2023-01-13 15:50:03,322 | server.py:

History (loss, distributed):
	round 1: 0.00047746858803445803
	round 2: 0.0007220059671171081
	round 3: 0.0007651691735494773
	round 4: 0.0003658641672275786
	round 5: 0.0005834240170316223